In [1]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.axes as axes
import copy
import random
import time
from random import uniform, choice
import numpy as np
from numpy import dot

In [2]:
class CellularAutomaton:
    
    Colors = {0:"gray", 1:"cyan"}
    
    def __init__(self, dimens):
        self.grid = [[0 for x in range(dimens[1])] for y in range(dimens[0])]
        self.width = dimens[0]
        self.height = dimens[1]
        
    def __eq__(self, other):
        return self.grid == other.grid
    
    def __neq__(self, other):
        return not self == other
    
    def __hash__(self):
        return hash(str(self.grid))%100000
    
    def __cmp__(self, other):
        if self == other:
            return 0
        return __cmp__(id(self), id(other))
    
    def __lt__(self, other):
        return False
    
    def clone(self):
        return copy.deepcopy(self)
    
    def draw(self, compare):
        fig1 = plt.figure()
        ax1 = fig1.add_subplot(1,2,1, aspect='equal')
        ax2 = fig1.add_subplot(1,2,2, aspect='equal')
        ax1.set_axis_bgcolor('gray')
        ax1.set_xlim([0,self.width])
        ax1.set_ylim([0,self.height])
        ax2.set_axis_bgcolor('gray')
        ax2.set_xlim([0,len(compare)])
        ax2.set_ylim([0,len(compare[0])])
        for x in range(0, self.width):
            for y in range(0, self.height):
                cell = self.grid[x][y]
                if cell == 0: continue
                ax1.add_patch(
                    patches.Rectangle((x, y),
                                      1,1,
                                      fill=True,
                                      facecolor=CellularAutomaton.Colors[cell],
                                      edgecolor="black",
                                      hatch= None,
                                      label=cell)
                    )
        for x in range(0, len(compare)):
            for y in range(0, len(compare[0])):
                cell = compare[x][y]
                if cell == 0: continue
                ax2.add_patch(
                    patches.Rectangle((x, y),
                                      1,1,
                                      fill=True,
                                      facecolor=CellularAutomaton.Colors[cell],
                                      edgecolor="black",
                                      hatch= None,
                                      label=cell)
                    )
        plt.show(fig1)
                

#example = CellularAutomaton((30,30))
#example.grid[2][2] = 1
#example.draw()

In [3]:
class GameOfLife(CellularAutomaton):
    
    def __init__(self, dimens):
        CellularAutomaton.__init__(self, dimens)
        self.turns = 0
            
    def num_neighbors(self, pos):
        neighbors = 0
        for i in range(-1,2):
            for j in range(-1,2):
                x = pos[0] + i
                y = pos[1] + j
                if (i is not 0 or j is not 0) and x >= 0 and y >= 0 and x < self.width and y < self.height:
                    if self.grid[x][y] is not 0:
                        neighbors += 1
        return neighbors
    
    def become_alive(self, pos):
        neighbors = self.num_neighbors(pos)
        if neighbors == 3:
            return True
        else:
            return False
    
    def check_alive(self, pos):
        neighbors = self.num_neighbors(pos)
        if neighbors < 2 or neighbors > 3:
            return False
        else:
            return True
    
    # 1 4 6
    # 2 0 7
    # 3 5 8
    def get_training_data(self, pos):
        neighbors = []
        neighbors.append(self.grid[pos[0]][pos[1]])
        for i in range(-1,2):
            for j in range(-1,2):
                x = pos[0] + i
                y = pos[1] + j
                if (i is not 0 or j is not 0):
                    if x < 0 or y < 0 or x >= self.width or y >= self.height or self.grid[x][y] is 0:
                        neighbors.append(0)
                    else:
                        neighbors.append(self.grid[x][y])
        return tuple(neighbors)
    
    def play(self):
        training = []
        new_game = self.clone()
        for x in range(0, self.width):
            for y in range(0, self.height):
                cell = self.grid[x][y]
                if cell == 0:
                    if self.become_alive((x,y)):
                        new_game.grid[x][y] = 1
                elif cell == 1:
                    if not self.check_alive((x,y)):
                        new_game.grid[x][y] = 0
                training.append((self.get_training_data((x,y)), new_game.grid[x][y]))
        
        self.grid = new_game.grid
        self.turns += 1
        return training
        

In [4]:
game = GameOfLife((20, 20))
#glider
game.grid[0][17] = 1
game.grid[1][17] = 1
game.grid[1][18] = 1
game.grid[2][18] = 1
game.grid[0][19] = 1
#blinker
game.grid[12][12] = 1
game.grid[12][11] = 1
game.grid[12][10] = 1

for x in range(0, game.width):
    for y in range(0, game.height):
        num = random.randint(0,6)
        if num == 0:
            game.grid[x][y] = 1
        else:
            game.grid[x][y] = 0

compare = [[0 for x in range(0,20)] for y in range(0,20)]
#game.draw(compare)


#for x in range(0, 10):
#    game.play()
#    game.draw(compare)

In [5]:
def tuple_append(tpl,elt):
    return tuple(list(tpl)+[elt])

def perceptron(training, epochs):
    rate = 0.5
    feature_count = len(training[0][0])
    
    training = [(tuple_append(ins,1), label) for (ins,label) in training]
    
    weights = [uniform(0,0.05) for _r in range(0,feature_count+1)]
    
    errors = []
    for i in range(0,epochs):
        # in each epoch, pick a subset of the training set (just one for now)
        (example,actual) = training[i]
        # calculate the perceptron activation and the predicted category
        # Note: You can use dot(vec1, vec2) to get the dot product between two sequential collections.
        activation = dot(example, weights)
        predicted = 0
        error = 0
        if activation >= 0 and activation < 1:
            predicted = 1
        error = actual-predicted
        # calculate the error between predicted and actual and add it to errors
        errors.append(error)
        # update each weight according to the perceptron update rule
        for i in range(0, feature_count+1):
            weights[i] += example[i]*error
    
    return (errors, weights)

In [7]:
#training = [((1,1,0,0,0,1,0,0,0), 0)]
epochs = 500

game = GameOfLife((20,20))
for x in range(0, game.width):
    for y in range(0, game.height):
        num = random.randint(0,5)
        if num == 0:
            game.grid[x][y] = 1
        else:
            game.grid[x][y] = 0

training = []
for i in range(0, epochs):
    if i%15 == 0:
        game = GameOfLife((20,20))
        for x in range(0, game.width):
            for y in range(0, game.height):
                num = random.randint(0,4)
                if num == 0:
                    game.grid[x][y] = 1
                else:
                    game.grid[x][y] = 0
    training += game.play()

(errors, weights) = perceptron(training, len(training))
print(weights)
percentage = 0.0
sum = 0.0
for x in range(len(errors)-100, len(errors)):
    sum += abs(errors[x])
print(errors)
percentage = sum/100.0
print(percentage)



[16877.036811794696, 9167.04634763876, 10102.01019371171, 9128.04055451586, 10198.00971775921, 10111.023270593632, 9293.030636358031, 10079.006068468947, 9172.003477513597, 28729.020451619992]
[-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0

In [29]:
from sknn.mlp import Classifier, Convolution, Layer

#generate game
epochs = 5
(w, h) = (10,10)
game = GameOfLife((w,h))
for x in range(0, game.width):
            for y in range(0, game.height):
                num = random.randint(0,4)
                if num == 0:
                    game.grid[x][y] = 1
                else:
                    game.grid[x][y] = 0
                    
#get training data
training = []
for i in range(0, 1):
    if i%30 == 0:
        game = GameOfLife((w,h))
        for x in range(0, game.width):
            for y in range(0, game.height):
                num = random.randint(0,4)
                if num == 0:
                    game.grid[x][y] = 1
                else:
                    game.grid[x][y] = 0
    training += game.play()

#initialize X_train inputs and y_train outputs
X_train = np.array([[0 for y in range(len(training[0][0]))] for x in range(len(training))])
y_train = np.array([[0, 0] for y in range(len(training))])
for x in range(len(training)):
    array = []
    stuff = list(training[x][0])
    for i in stuff:
        array.append(i)
    X_train[x] = array
    y_train[x][training[x][1]] = 1

#create a neural network
nn = Classifier(
    layers=[
        Convolution("Rectifier", channels=1, kernel_shape=(3,3)),
        Layer("Softmax")],
    learning_rate=0.02,
    n_iter=25)

game.draw(game.grid)
#debugging
print("----------")
print(X_train)
print("---------")
print(y_train)
print("------------")
nn.fit(X_train, y_train)



----------
[[0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 1 0 1 1]
 [1 0 0 0 0 1 1 1 1]
 [1 0 0 0 1 0 1 1 0]
 [0 0 0 0 1 0 1 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 1 0 1 0]
 [1 0 0 1 0 1 1 0 0]
 [1 0 1 1 1 1 0 0 0]
 [1 1 1 0 1 0 0 0 0]
 [0 1 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0]
 [1 0 0 1 0 0 0 0 0]
 [0 0 1 1 1 0 0 0 0]
 [0 1 1 1 0 0 0 0 0]
 [0 1 1 0 0 0 0 0 1]
 [0 1 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 1 0 1]
 [0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 1 1]
 [0 0 0 0 0 1 1 1 0]
 [1 0 0 0 0 0 1 0 0]
 [0 0 0 0 1 1 0 0 0]
 [1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 1]
 [1 0 0 0 0 1 0 1 0]
 [1 0 0 1 1 0 1 0 0]
 [

C:\Users\Luke\Anaconda3\lib\site-packages\lasagne\layers\conv.py:489: UserWarning: The `image_shape` keyword argument to `tensor.nnet.conv2d` is deprecated, it has been renamed to `input_shape`.
  border_mode=border_mode)


Classifier(batch_size=1, callback=None, debug=False, dropout_rate=None,
      f_stable=0.001,
      hidden0=<sknn.nn.Convolution `Rectifier`: scale_factor=(1, 1), kernel_stride=(1, 1), border_mode='valid', kernel_shape=(3, 3), channels=1, name='hidden0', frozen=False, pool_shape=(1, 1)>,
      layers=[<sknn.nn.Convolution `Rectifier`: scale_factor=(1, 1), kernel_stride=(1, 1), border_mode='valid', kernel_shape=(3, 3), channels=1, name='hidden0', frozen=False, pool_shape=(1, 1)>, <sknn.nn.Layer `Softmax`: name='output', frozen=False, units=4>],
      learning_momentum=0.9, learning_rate=0.02, learning_rule='sgd',
      loss_type=None, n_iter=25, n_stable=10, normalize=None,
      output=<sknn.nn.Layer `Softmax`: name='output', frozen=False, units=4>,
      parameters=None, random_state=None, regularize=None, valid_set=None,
      valid_size=0.0, verbose=None, warning=None, weight_decay=None)

In [30]:
#get testing data
testing = []
for i in range(0, 1):
    if i%30 == 0:
        game = GameOfLife((w,h))
        for x in range(0, game.width):
            for y in range(0, game.height):
                num = random.randint(0,4)
                if num == 0:
                    game.grid[x][y] = 1
                else:
                    game.grid[x][y] = 0
    testing += game.play()

#initialize X_test
X_test = np.array([[0 for x in range(len(testing[0][0]))] for y in range(len(testing))])
for x in testing:
    array = []
    for i in x[0]:
        array.append(i)
    np.append(X_test, array)
y_test = nn.predict(X_test)
print(y_test)

[(100, 2), (100, 2)]
[[1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]]


In [ ]:
inputs["channels(1 for grayscale)", size=1][columns, size=width][rows, size=height]